In [ ]:
!pip install transformers -U
!pip install accelerate -U
!pip install -U flash-attn --no-build-isolation #install flash attention

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModel
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import time

In [ ]:
# Load the dataset
from google.colab import drive
drive.mount('/content/drive')
java_file_path = '/content/drive/MyDrive/Wajid Ali/FYP/Dataset/java.csv'
python_file_path = '/content/drive/MyDrive/Wajid Ali/FYP/Dataset/python.csv'
pharo_file_path = '/content/drive/MyDrive/Wajid Ali/FYP/Dataset/pharo.csv'
java_df = pd.read_csv(java_file_path)
python_df = pd.read_csv(python_file_path)
pharo_df = pd.read_csv(pharo_file_path)

In [ ]:
# Data preparation
java_df['category'] = java_df['category'].replace('Expand', 'java_Expand')
df = pd.concat([java_df, python_df, pharo_df], ignore_index=True)

In [ ]:
# Preprocessing of the dataset
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess_text(text):
    # Remove extra spaces while allowing all characters
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
df['combo'] = df['class'] + " " + df['comment_sentence']
df['combo'] = df['combo'].apply(preprocess_text)

In [ ]:
# Split the dataset into training and testing data
train_data = df[df['partition'] == 1]
test_data = df[df['partition'] == 0]

In [ ]:
# Tokenization and Dataset preparation
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)


In [ ]:
class CommentDataset(Dataset):
    def __init__(self, combos, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.combos = combos
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.combos)

    def __getitem__(self, idx):
        combo = str(self.combos[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            combo,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
train_dataset = CommentDataset(train_data['combo'].to_numpy(), train_data['category'].astype('category').cat.codes.to_numpy(), tokenizer)
test_dataset = CommentDataset(test_data['combo'].to_numpy(), test_data['category'].astype('category').cat.codes.to_numpy(), tokenizer)

In [ ]:
# Training the model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps'
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()